In [1]:
import numpy as np 
import torch

In [ ]:
grad_states = np.load('./images/test_training/runs514/grad_states.npy', allow_pickle=True)
print(f'grad_states.len: {len(grad_states)}')
step_size = 1/pow(2, 13)
ceil = pow(2, 0)
for bi in range(len(grad_states)):
    if bi == 0:
        print(f'param.names.len: {grad_states[bi].keys()}')

    gmin = 1e10 if bi % 40 ==0 else gmin
    gmax = 1e-10 if bi % 40 ==0 else gmax
    cnt_higherer_ceil = 0 if bi % 40 ==0 else cnt_higherer_ceil
    cnt_lower_stepsize = 0 if bi % 40 ==0 else cnt_lower_stepsize
    cnt_total = 0 if bi % 40 ==0 else cnt_total
    for key, grad in grad_states[bi].items():
        grad_abs = abs(grad).reshape(-1)
        step_size_ = step_size * np.ones(grad_abs.shape[0])

        cnt_total += grad_abs.shape[0]
        cnt_higherer_ceil += sum(grad_abs > ceil)
        cnt_lower_stepsize += sum(grad_abs <= step_size_)
        gmin = min(gmin, abs(grad).min())
        gmax = max(gmax, abs(grad).max())
    
    if bi % 40 == 0:
        print(f'bi: {bi}, gmin: {gmin}, gmax: {gmax}')
        print(f'bi: {bi}, {(100*cnt_higherer_ceil)/cnt_total:.2f}% higher than the ceil, {(100*cnt_lower_stepsize)/cnt_total:.2f}% lower than the step_size')

grad_states.len: 40
param.names.len: dict_keys(['features.0.0.full_precision_weight', 'features.1.0.block.0.0.full_precision_weight', 'features.1.0.block.1.fc1.full_precision_weight', 'features.1.0.block.1.fc2.full_precision_weight', 'features.1.0.block.2.0.full_precision_weight', 'features.2.0.block.0.0.full_precision_weight', 'features.2.0.block.1.0.full_precision_weight', 'features.2.0.block.2.fc1.full_precision_weight', 'features.2.0.block.2.fc2.full_precision_weight', 'features.2.0.block.3.0.full_precision_weight', 'features.2.1.block.0.0.full_precision_weight', 'features.2.1.block.1.0.full_precision_weight', 'features.2.1.block.2.fc1.full_precision_weight', 'features.2.1.block.2.fc2.full_precision_weight', 'features.2.1.block.3.0.full_precision_weight', 'features.3.0.block.0.0.full_precision_weight', 'features.3.0.block.1.0.full_precision_weight', 'features.3.0.block.2.fc1.full_precision_weight', 'features.3.0.block.2.fc2.full_precision_weight', 'features.3.0.block.3.0.full_preci

In [14]:
names = list(grad_states[0].keys())
print(len(names))
for name in names:
    assert name.split('.')[-1] == "full_precision_weight"

name = 'features.0.0.full_precision_weight'
print(grad_states[0][name])
print(grad_states[0][name].min(), grad_states[0][name].max())

82
[[[[-4.00848314e-02 -3.87345739e-02 -3.93881984e-02]
   [-4.41481061e-02 -4.06590588e-02 -4.06205244e-02]
   [-4.95708324e-02 -4.48223613e-02 -4.30317856e-02]]

  [[-1.98466126e-02 -1.53302774e-02 -1.62526201e-02]
   [-1.67152472e-02 -1.46640809e-02 -1.57606732e-02]
   [-1.70365907e-02 -1.81943402e-02 -1.77188683e-02]]

  [[ 4.70865238e-03  7.61942193e-03  2.09176866e-03]
   [ 4.71205777e-03  5.83358621e-03  5.39171603e-03]
   [ 1.73996529e-03  1.34591805e-03  4.59037023e-03]]]


 [[[ 1.05659151e-02  1.09349852e-02  1.28682964e-02]
   [ 1.05371336e-02  1.16112055e-02  1.18030598e-02]
   [ 8.86879396e-03  1.10607930e-02  1.19861085e-02]]

  [[-1.53862871e-02 -1.48972264e-02 -1.66981649e-02]
   [-1.33345956e-02 -1.48230996e-02 -1.36155477e-02]
   [-1.48553718e-02 -1.59499794e-02 -1.34467855e-02]]

  [[-7.38703273e-03 -6.24962058e-03 -7.00069638e-03]
   [-6.40980527e-03 -4.96395817e-03 -4.05375613e-03]
   [-6.85863802e-03 -7.30753923e-03 -4.64291917e-03]]]


 [[[-1.31205637e-02 -1.3363

In [15]:
n_layers = 10
step_size = 1/pow(2, 14)
for bi in range(len(grad_states)):
    if bi > 40 :
        break
    states = {}
    layer_pre = 0
    cnt, cnt_total = 0, 0
    for name, grad in grad_states[bi].items():
        feature = name.split('.')[0]
        layer = int(name.split('.')[1])
        if feature == 'classifier':
            layer = 9
        if layer != layer_pre:
            print(f'layer: {layer_pre}, {(100*cnt)/cnt_total:.4f}% lower than the step_size')
            cnt, cnt_total = 0, 0
            layer_pre = layer

        type = '.'.join(name.split('.')[-2:])
        if type == '0.weight':
            grad_abs = abs(grad).reshape(-1)
            step_size_ = step_size * np.ones(grad_abs.shape[0])
            cnt_total += grad_abs.shape[0]
            cnt += sum(grad_abs <= step_size_)


ZeroDivisionError: division by zero

In [ ]:
from train_quantization import RoundWrapper, QuantizedWrapper, ResnetQAT, EfficientnetQAT
d = 'cuda' if torch.cuda.is_available() else 'cpu'
model = torch.load('./images/test_training/runs748/epoch500_tacc0.995_vacc0.718.ckp', map_location=d)
ms = list(model.named_children())

3
